In [1]:
import networkx as nx
from node2vec import Node2Vec
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import numpy as np

In [2]:
node = []
edge_train = []
edge_test = []
edge_exit = []
count = 0
with open('facebook_combined.txt', 'r') as f:
    data = f.readlines()
    for line in data:
        line = tuple(line.replace('\n', '').split(' '))
        edge_exit.append(line)
        
        if line[0] not in node:
            node.append(line[0])
        
        if line[1] not in node:
            node.append(line[1])
            
        if(count%5 == 0):
            edge_test.append(line)
            count+=1
        else:
            edge_train.append(line)
            count+=1

In [3]:
G = nx.DiGraph()
G.add_nodes_from(node)
G.add_edges_from(edge_train)

In [4]:
G_all = nx.DiGraph()
G_all.add_nodes_from(node)
G_all.add_edges_from(edge_exit)

In [5]:
edge_not_exit = list(nx.non_edges(G_all))

print(len(edge_train))
print(len(edge_test))
print(len(edge_not_exit))

70587
17647
16221248


In [6]:
node2vec = Node2Vec(G, dimensions = 128, walk_length = 80, num_walks = 10, p = 0.3, q = 0.7, workers = 4)

Computing transition probabilities:   0%|          | 0/4039 [00:00<?, ?it/s]

Generating walks (CPU: 4): 100%|██████████| 2/2 [00:01<00:00,  1.61it/s]

In [7]:
model = node2vec.fit()

In [18]:
# len_not = len(edge_not_exit)
# len_test = len(edge_test)
# score_total = 0.0
# times = 0.0
# for i in range(len_test):
#     sim_test = model.wv.similarity(edge_test[i][0], edge_test[i][1])
#     j = 0
#     while(j < len_not):
#         sim_not = model.wv.similarity(edge_not_exit[j][0], edge_not_exit[j][1])
#         times += 1.0
#         j += 15000
#         if(sim_test > sim_not):
#             score_total += 1.0
#         elif(sim_test == sim_not):
#             score_total += 0.5
#         else:
#             score_total += 0.0
            
# #     for j in range(int(len_not/10000)):
# #         sim_not = model.wv.similarity(edge_not_exit[j][0], edge_not_exit[j][1])
# #         times += 1.0
# #         if(sim_test > sim_not):
# #             score_total += 1.0
# #         elif(sim_test == sim_not):
# #             score_total += 0.5
# #         else:
# #             score_total += 0.0

# score = score_total/ times
# print(score)

In [10]:
print(model.wv.most_similar('2', topn=5))

[('115', 0.9935605525970459), ('116', 0.9932399988174438), ('20', 0.992276132106781), ('28', 0.991713285446167), ('44', 0.9912866950035095)]


In [20]:
# print(model.wv.most_similar('2', topn=5))
# print(model.wv.similarity('0', '1'))
# print(model.wv.similarity(edge_test[0][0], edge_test[0][1]))

In [48]:
len_not = len(edge_not_exit)
len_test = len(edge_test)
y_true = np.array([])
y_scores = np.array([])

for i in range(len_test):
    y_true = np.append(y_true, 1)
    sim = model.wv.similarity(edge_test[i][0], edge_test[i][1])
    y_scores = np.append(y_scores, sim)

In [53]:
j=0
while(j < len_not):
    y_true = np.append(y_true, 0)
    sim = model.wv.similarity(edge_not_exit[i][0], edge_not_exit[i][1])
    y_scores = np.append(y_scores, sim)
    j += 300

In [55]:
print(roc_auc_score(y_true, y_scores))

0.9809599365331217
